# 分组
 

In [1]:
import numpy as np
import pandas as pd
pd.__version__

'1.4.0'

In [123]:
df = pd.read_csv('data/table.csv',index_col='ID') # 将 ID 列设置为行索引
df.head()

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1104,S_1,C_1,F,street_2,167,81,80.4,B-
1105,S_1,C_1,F,street_4,159,64,84.8,B+


In [124]:
df.shape

(35, 8)

## 一、SAC过程
### 1. 内涵
#### SAC指的是分组操作中的split-apply-combine过程
#### 其中split指基于某一些规则，将数据拆成若干组，apply是指对每一组独立地使用函数，combine指将每一组的结果组合成某一类数据结构
### 2. apply过程
#### 在该过程中，我们实际往往会遇到四类问题：
#### 整合（Aggregation）——即分组计算统计量（如求均值、求每组元素个数）
#### 变换（Transformation）——即分组对每个单元的数据进行操作（如元素标准化）
#### 过滤（Filtration）——即按照某些规则筛选出一些组（如选出组内某一指标小于50的组）
#### 综合问题——即前面提及的三种问题的混合

## 二、groupby函数


### 1. 分组函数的基本内容：

#### （a）根据某一列分组

In [36]:
grouped_single = df.groupby('School')
grouped_single

#### 经过groupby后会生成一个groupby对象，该对象本身不会返回任何东西，只有当相应的方法被调用才会起作用
#### 例如取出某一个组：

In [37]:
grouped_single.get_group('S_1').head()
# 注意这时分组列并未被作为索引加进去

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1104,S_1,C_1,F,street_2,167,81,80.4,B-
1105,S_1,C_1,F,street_4,159,64,84.8,B+


In [20]:
# grouped_single.get_group 方法可以按照分组列的不同取值取回不同的子df
grouped_single.get_group??
#  Construct DataFrame from group with provided name.

Signature: grouped_single.get_group(name, obj=None)
Source:   
    def get_group(self, name, obj=None):
        """
        Construct DataFrame from group with provided name.

        Parameters
        ----------
        name : object
            The name of the group to get as a DataFrame.
        obj : DataFrame, default None
            The DataFrame to take the DataFrame out of.  If
            it is None, the object groupby was called on will
            be used.

        Returns
        -------
        group : same type as obj
        """
        if obj is None:
            obj = self._selected_obj

        inds = self._get_index(name)
        if not len(inds):
            raise KeyError(name)

        return obj._take_with_is_copy(inds, axis=self.axis)
File:      d:\programdata\anaconda3\lib\site-packages\pandas\core\groupby\groupby.py
Type:      method


In [42]:
# 等价的方法是?
[groups[1].head() for groups in grouped_single if groups[0]=='S_1' ]

[     School Class Gender   Address  Height  Weight  Math Physics
 ID                                                              
 1101    S_1   C_1      M  street_1     173      63  34.0      A+
 1102    S_1   C_1      F  street_2     192      73  32.5      B+
 1103    S_1   C_1      M  street_2     186      82  87.2      B+
 1104    S_1   C_1      F  street_2     167      81  80.4      B-
 1105    S_1   C_1      F  street_4     159      64  84.8      B+]

In [45]:
# 等价的方法
for groups in grouped_single:
    if groups[0]=='S_1':
        display(groups[1].head())
        print(type(groups[1]))

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1104,S_1,C_1,F,street_2,167,81,80.4,B-
1105,S_1,C_1,F,street_4,159,64,84.8,B+


<class 'pandas.core.frame.DataFrame'>


In [47]:
display(grouped_single)
# 并不能被显示
print(grouped_single)

In [48]:
# groupby得到的其实是把原始df 按照分组拆分成的几个子df及其名称(其名称是分组列中的不同取值),支持遍历
for groups in grouped_single:
    print(groups[0])
    print(type(groups[0]))
    print(groups[1].head(2))
    print(type(groups[1]))

S_1
<class 'str'>
     School Class Gender   Address  Height  Weight  Math Physics
ID                                                              
1101    S_1   C_1      M  street_1     173      63  34.0      A+
1102    S_1   C_1      F  street_2     192      73  32.5      B+
<class 'pandas.core.frame.DataFrame'>
S_2
<class 'str'>
     School Class Gender   Address  Height  Weight  Math Physics
ID                                                              
2101    S_2   C_1      M  street_7     174      84  83.3       C
2102    S_2   C_1      F  street_6     161      61  50.6      B+
<class 'pandas.core.frame.DataFrame'>


In [52]:
# grouped_single = df.groupby('School') 的本质是把df按照School列的不同取值拆分为多个df, 每个子df分别用School相应的不同取值命名.
names_of_sub_df=list(df.School.unique())
grouped_by_School=[]
for name in names_of_sub_df:
    sub_df=df[df.School==name]
    grouped_by_School.append([name,sub_df])
grouped_by_School

[['S_1',      School Class Gender   Address  Height  Weight  Math Physics
  ID                                                              
  1101    S_1   C_1      M  street_1     173      63  34.0      A+
  1102    S_1   C_1      F  street_2     192      73  32.5      B+
  1103    S_1   C_1      M  street_2     186      82  87.2      B+
  1104    S_1   C_1      F  street_2     167      81  80.4      B-
  1105    S_1   C_1      F  street_4     159      64  84.8      B+
  1201    S_1   C_2      M  street_5     188      68  97.0      A-
  1202    S_1   C_2      F  street_4     176      94  63.5      B-
  1203    S_1   C_2      M  street_6     160      53  58.8      A+
  1204    S_1   C_2      F  street_5     162      63  33.8       B
  1205    S_1   C_2      F  street_6     167      63  68.4      B-
  1301    S_1   C_3      M  street_4     161      68  31.5      B+
  1302    S_1   C_3      F  street_1     175      57  87.7      A-
  1303    S_1   C_3      M  street_7     188      82  4

In [53]:
# 用同样的代码遍历 grouped_by_School: 
for groups in grouped_by_School:
    print(groups[0])
    print(type(groups[0]))
    print(groups[1].head(2))
    print(type(groups[1]))

S_1
<class 'str'>
     School Class Gender   Address  Height  Weight  Math Physics
ID                                                              
1101    S_1   C_1      M  street_1     173      63  34.0      A+
1102    S_1   C_1      F  street_2     192      73  32.5      B+
<class 'pandas.core.frame.DataFrame'>
S_2
<class 'str'>
     School Class Gender   Address  Height  Weight  Math Physics
ID                                                              
2101    S_2   C_1      M  street_7     174      84  83.3       C
2102    S_2   C_1      F  street_6     161      61  50.6      B+
<class 'pandas.core.frame.DataFrame'>


#### （b）根据某几列分组

In [54]:
# 根据多列分组, 实现效果类似于根据多列的不同组合进行分组--虽然本质上并不一定是这样的,但可以这么理解.
grouped_mul = df.groupby(['School','Class'])
grouped_mul.get_group(('S_2','C_4'))
# 按多列分组, 取某一组时, 需要传入tuple

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
2401,S_2,C_4,F,street_2,192,62,45.3,A
2402,S_2,C_4,M,street_7,166,82,48.7,B
2403,S_2,C_4,F,street_6,158,60,59.7,B+
2404,S_2,C_4,F,street_2,160,84,67.7,B
2405,S_2,C_4,F,street_6,193,54,47.6,B


In [55]:
grouped_mul.get_group(['S_2','C_4'])
# ValueError: must supply a tuple to get_group with multiple grouping keys
# 传入list是不行的

ValueError: must supply a tuple to get_group with multiple grouping keys

#### （c）组容量与组数

In [56]:
# 组容量就是每组所包含的行数
grouped_single.size()

School
S_1    15
S_2    20
dtype: int64

In [58]:
grouped_mul.size()

School  Class
S_1     C_1      5
        C_2      5
        C_3      5
S_2     C_1      5
        C_2      5
        C_3      5
        C_4      5
dtype: int64

In [61]:
grouped_mul.count()

Gender  Address  Height  Weight  Math  Physics
School Class                                                
S_1    C_1         5        5       5       5     5        5
       C_2         5        5       5       5     5        5
       C_3         5        5       5       5     5        5
S_2    C_1         5        5       5       5     5        5
       C_2         5        5       5       5     5        5
       C_3         5        5       5       5     5        5
       C_4         5        5       5       5     5        5

In [59]:
# 组数就是分了几组
grouped_single.ngroups

2

In [60]:
# 按多列分组,则统计的是两列的不同组合数
grouped_mul.ngroups

7

In [63]:
len(grouped_mul)

7

#### （d）组的遍历

In [64]:
for name,group in grouped_single:
    print(name)
    display(group.head(3))

S_1


,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+


S_2


,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
2101,S_2,C_1,M,street_7,174,84,83.3,C
2102,S_2,C_1,F,street_6,161,61,50.6,B+
2103,S_2,C_1,M,street_4,157,61,52.5,B-


#### （e）level参数（用于多级索引）和axis参数

In [69]:
# 按多重索引的第二层索引分组
df.set_index(['Gender','School']).groupby(level=1,axis=0).get_group('S_1')#.head()

Class   Address  Height  Weight  Math Physics
Gender School                                              
M      S_1      C_1  street_1     173      63  34.0      A+
F      S_1      C_1  street_2     192      73  32.5      B+
M      S_1      C_1  street_2     186      82  87.2      B+
F      S_1      C_1  street_2     167      81  80.4      B-
       S_1      C_1  street_4     159      64  84.8      B+
M      S_1      C_2  street_5     188      68  97.0      A-
F      S_1      C_2  street_4     176      94  63.5      B-
M      S_1      C_2  street_6     160      53  58.8      A+
F      S_1      C_2  street_5     162      63  33.8       B
       S_1      C_2  street_6     167      63  68.4      B-
M      S_1      C_3  street_4     161      68  31.5      B+
F      S_1      C_3  street_1     175      57  87.7      A-
M      S_1      C_3  street_7     188      82  49.7       B
       S_1      C_3  street_2     195      70  85.2       A
F      S_1      C_3  street_5     187      69  61.7      B-

In [70]:
# 按照第二层索引分组, 就会按照第二层索引的取值, 将取值相同的分拆出来,然后合并到一起---这就导致了索引未排序的效果.
df.set_index(['Gender','School']).loc[df.set_index(['Gender','School']).index.get_level_values(1)=='S_1'] 
#多重索引的 index.get_level_values 方法, 按照指定的索引层级, 筛选等于相应值的行--实现该效果有多种方法,例如query,IndexSlice等等

Class   Address  Height  Weight  Math Physics
Gender School                                              
M      S_1      C_1  street_1     173      63  34.0      A+
F      S_1      C_1  street_2     192      73  32.5      B+
M      S_1      C_1  street_2     186      82  87.2      B+
F      S_1      C_1  street_2     167      81  80.4      B-
       S_1      C_1  street_4     159      64  84.8      B+
M      S_1      C_2  street_5     188      68  97.0      A-
F      S_1      C_2  street_4     176      94  63.5      B-
M      S_1      C_2  street_6     160      53  58.8      A+
F      S_1      C_2  street_5     162      63  33.8       B
       S_1      C_2  street_6     167      63  68.4      B-
M      S_1      C_3  street_4     161      68  31.5      B+
F      S_1      C_3  street_1     175      57  87.7      A-
M      S_1      C_3  street_7     188      82  49.7       B
       S_1      C_3  street_2     195      70  85.2       A
F      S_1      C_3  street_5     187      69  61.7      B-

In [72]:
# 直接按多重索引分组
df_=df.set_index(['Gender','School'])
df_.groupby(df_.index).head(2)
# 按索引分组, 和按列分组并无不同, 都是将相应的列或者索引传给groupby
# 但是注意显示的结果是索引未排序的, 并且用于分组的索引也是显示了的--这和直接用loc方法得到的显示结果并不一样

,,Class,Address,Height,Weight,Math,Physics
Gender,School,,,,,,
M,S_1,C_1,street_1,173,63,34.0,A+
F,S_1,C_1,street_2,192,73,32.5,B+
M,S_1,C_1,street_2,186,82,87.2,B+
F,S_1,C_1,street_2,167,81,80.4,B-
M,S_2,C_1,street_7,174,84,83.3,C
F,S_2,C_1,street_6,161,61,50.6,B+
M,S_2,C_1,street_4,157,61,52.5,B-
F,S_2,C_1,street_5,159,97,72.2,B+


In [74]:
df_.loc['M'].head()
# 注意, 第一层索引并没有显示

,Class,Address,Height,Weight,Math,Physics
School,,,,,,
S_1,C_1,street_1,173,63,34.0,A+
S_1,C_1,street_2,186,82,87.2,B+
S_1,C_2,street_5,188,68,97.0,A-
S_1,C_2,street_6,160,53,58.8,A+
S_1,C_3,street_4,161,68,31.5,B+


### 2. groupby对象的特点

#### （a）查看所有可调用的方法

#### 由此可见，groupby对象可以使用相当多的函数，灵活程度很高

In [75]:
print([attr for attr in dir(grouped_single) if not attr.startswith('_') and attr not in df.columns])

['agg', 'aggregate', 'all', 'any', 'apply', 'backfill', 'bfill', 'boxplot', 'corr', 'corrwith', 'count', 'cov', 'cumcount', 'cummax', 'cummin', 'cumprod', 'cumsum', 'describe', 'diff', 'dtypes', 'expanding', 'ffill', 'fillna', 'filter', 'first', 'get_group', 'groups', 'head', 'hist', 'idxmax', 'idxmin', 'indices', 'last', 'mad', 'max', 'mean', 'median', 'min', 'ndim', 'ngroup', 'ngroups', 'nth', 'nunique', 'ohlc', 'pad', 'pct_change', 'pipe', 'plot', 'prod', 'quantile', 'rank', 'resample', 'rolling', 'sem', 'shift', 'size', 'skew', 'std', 'sum', 'tail', 'take', 'transform', 'tshift', 'var']


In [ ]:
# 一种直观但实际上并不是这样的看法是, 把groupby得到的对象当作是一个"特殊"的DF--索引为用于分组列的唯一值(或唯一的组合--对于按多列分组),"行"是一个子df.

#### （b）分组对象的head和first
#### 对分组对象使用head函数，返回的是每个组的前几行，而不是数据集前几行

In [76]:
grouped_single.head(2)
# 实际上是把head方法传递给了每一个子df

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
2101,S_2,C_1,M,street_7,174,84,83.3,C
2102,S_2,C_1,F,street_6,161,61,50.6,B+


In [78]:
grouped_single.head??
# Return first n rows of each group.
# Similar to ``.apply(lambda x: x.head(n))``
# but it returns a subset of rows from the original DataFrame with original index and order preserved (``as_index`` flag is ignored).

Signature: grouped_single.head(n=5)
Docstring:
Return first n rows of each group.

Similar to ``.apply(lambda x: x.head(n))``, but it returns a subset of rows
from the original DataFrame with original index and order preserved
(``as_index`` flag is ignored).

Does not work for negative values of `n`.

Returns
-------
Series or DataFrame

See Also
--------
Series.groupby
DataFrame.groupby

Examples
--------

>>> df = pd.DataFrame([[1, 2], [1, 4], [5, 6]],
...                   columns=['A', 'B'])
>>> df.groupby('A').head(1)
   A  B
0  1  2
2  5  6
>>> df.groupby('A').head(-1)
Empty DataFrame
Columns: [A, B]
Index: []
Source:   
    @Substitution(name="groupby")
    @Substitution(see_also=_common_see_also)
    def head(self, n=5):
        """
        Return first n rows of each group.

        Similar to ``.apply(lambda x: x.head(n))``, but it returns a subset of rows
        from the original DataFrame with original index and order preserved
        (``as_index`` flag is ignored).

    

In [79]:
# 在筛选行的意义上等价的applay方式
grouped_single.apply(lambda x: x.head(2))
# 但是注意把School也设置成了索引,并且是顶层索引--这是因为grouped_single就是按照Sschool进行分组的

Class Gender   Address  Height  Weight  Math Physics
School ID                                                       
S_1    1101   C_1      M  street_1     173      63  34.0      A+
       1102   C_1      F  street_2     192      73  32.5      B+
S_2    2101   C_1      M  street_7     174      84  83.3       C
       2102   C_1      F  street_6     161      61  50.6      B+

In [82]:
# 作为对比,原始df及分组后的group对象中的子df,都是以ID为索引的
grouped_single.get_group('S_1').head()

,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,
1101,C_1,M,street_1,173,63,34.0,A+
1102,C_1,F,street_2,192,73,32.5,B+
1103,C_1,M,street_2,186,82,87.2,B+
1104,C_1,F,street_2,167,81,80.4,B-
1105,C_1,F,street_4,159,64,84.8,B+


#### first显示的是以分组为索引的每组的第一个分组信息

In [85]:
grouped_single.first()
# 注意原始的索引 ID 被丢弃了

,Class,Gender,Address,Height,Weight,Math,Physics
School,,,,,,,
S_1,C_1,M,street_1,173,63,34.0,A+
S_2,C_1,M,street_7,174,84,83.3,C


In [86]:
grouped_single.first??
#     Computed first of values within each group.

Signature: grouped_single.first(**kwargs)
Docstring:
Compute first of group values.

Returns
-------
Series or DataFrame
    Computed first of values within each group.

        See Also
        --------
        Series.groupby
        DataFrame.groupby
Source:   
            @Substitution(name="groupby", f=name)
            @Appender(_common_see_also)
            @Appender(_local_template)
            def f(self, **kwargs):
                if "numeric_only" not in kwargs:
                    kwargs["numeric_only"] = numeric_only
                if "min_count" not in kwargs:
                    kwargs["min_count"] = min_count

                self._set_group_selection()

                # try a cython aggregation if we can
                try:
                    return self._cython_agg_general(alias, alt=npfunc, **kwargs)
                except DataError:
                    pass
                except NotImplementedError as err:
                    if "function is not implemented for 

In [87]:
# 效果上等价于: 对原始df先按照School去重,保留第一个不重复项, 然后将School设置为索引,并丢弃原始索引
df.drop_duplicates('School',keep='first').set_index('School',drop=True)

,Class,Gender,Address,Height,Weight,Math,Physics
School,,,,,,,
S_1,C_1,M,street_1,173,63,34.0,A+
S_2,C_1,M,street_7,174,84,83.3,C


#### （c）分组依据
#### 对于groupby函数而言，分组的依据是非常自由的，只要是与数据框长度相同的列表即可，同时支持函数型分组

In [88]:
df.groupby(np.random.choice(['a','b','c'],df.shape[0])).get_group('a').head()
#相当于将np.random.choice(['a','b','c'],df.shape[0])当做新的一列进行分组
# 该方法实际上也顺便实现了随机抽样的功能: 
##根据抽样比例,例如(无放回)抽取总体的三分之一,则给方法 np.random.choice 传入三个不同的值(a,b,c)组成的list, 长度设置为与df等长, 然后选取任意一个值(例如a)对应的分组

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1105,S_1,C_1,F,street_4,159,64,84.8,B+
1201,S_1,C_2,M,street_5,188,68,97.0,A-
1202,S_1,C_2,F,street_4,176,94,63.5,B-


#### 从原理上说，我们可以看到利用函数时，传入的对象就是索引，因此根据这一特性可以做一些复杂的操作

In [92]:
df[:5].groupby(lambda x:print(x)).head()
# 

1101
1102
1103
1104
1105


,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1104,S_1,C_1,F,street_2,167,81,80.4,B-
1105,S_1,C_1,F,street_4,159,64,84.8,B+


In [94]:
df[:5].groupby(lambda x:x).head()

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1104,S_1,C_1,F,street_2,167,81,80.4,B-
1105,S_1,C_1,F,street_4,159,64,84.8,B+


In [95]:
df[:5].groupby(lambda x:print(1)).head()

1
1
1
1
1


,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1104,S_1,C_1,F,street_2,167,81,80.4,B-
1105,S_1,C_1,F,street_4,159,64,84.8,B+


#### 根据奇偶行分组

In [120]:
df.groupby(lambda x:'奇数行' if not df.index.get_loc(x)%2==1 else '偶数行').groups

{'偶数行': Int64Index([1], dtype='int64'),
 '奇数行': Int64Index([0, 2], dtype='int64')}

In [126]:
# 本质上是新增了一个列, 列值用lambda函数赋值, 然后按这个新增的列分组
for groups in df.groupby(lambda x:'奇数行' if not df.index.get_loc(x)%2==1 else '偶数行'):
    print(groups[0])
    display(groups[1].head(2))

偶数行


,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1104,S_1,C_1,F,street_2,167,81,80.4,B-


奇数行


,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1103,S_1,C_1,M,street_2,186,82,87.2,B+


#### 如果是多层索引，那么lambda表达式中的输入就是元组，下面实现的功能为查看两所学校中男女生分别均分是否及格
#### 注意：此处只是演示groupby的用法，实际操作不会这样写

In [129]:
math_score = df.set_index(['Gender','School'])['Math'].sort_index()
math_score.head()

Gender  School
F       S_1       32.5
        S_1       80.4
        S_1       84.8
        S_1       63.5
        S_1       33.8
Name: Math, dtype: float64

In [141]:
math_score.groupby(math_score.index).mean()

(F, S_1)    64.100000
(F, S_2)    66.427273
(M, S_1)    63.342857
(M, S_2)    51.155556
Name: Math, dtype: float64

In [142]:
grouped_score = df.set_index(['Gender','School']).sort_index().\
            groupby(lambda x:(x,'Math均分及格' if math_score[x].mean()>=60 else 'Math均分不及格')) # 这里的x其实是'Gender','School'构成的层次索引
type(grouped_score)

pandas.core.groupby.generic.DataFrameGroupBy

In [137]:
for name,_ in grouped_score:
    print(name,_['Math'].mean())
    display(_.head(2))

(('F', 'S_1'), 'Math均分及格') 64.1


Class   Address  Height  Weight  Math Physics
Gender School                                              
F      S_1      C_1  street_2     192      73  32.5      B+
       S_1      C_1  street_2     167      81  80.4      B-

(('F', 'S_2'), 'Math均分及格') 66.42727272727274


Class   Address  Height  Weight  Math Physics
Gender School                                              
F      S_2      C_1  street_6     161      61  50.6      B+
       S_2      C_1  street_5     159      97  72.2      B+

(('M', 'S_1'), 'Math均分及格') 63.34285714285714


Class   Address  Height  Weight  Math Physics
Gender School                                              
M      S_1      C_1  street_1     173      63  34.0      A+
       S_1      C_1  street_2     186      82  87.2      B+

(('M', 'S_2'), 'Math均分不及格') 51.155555555555544


Class   Address  Height  Weight  Math Physics
Gender School                                              
M      S_2      C_1  street_7     174      84  83.3       C
       S_2      C_1  street_4     157      61  52.5      B-

In [144]:
# 如果不做变换, 就相当于为分组方法多加了一个附带标记,但由于这个附带标记是从分组方法导出的,因此增加这个附带标记并不会改变分组效果.
grouped_score_1 = df.set_index(['Gender','School']).sort_index().\
            groupby(lambda x:(x,math_score[x].mean())) # 这里的x其实是'Gender','School'构成的层次索引
for name,_ in grouped_score_1:
    print(name,_['Math'].mean())
    display(_.head(2))

(('F', 'S_1'), 64.1) 64.1


Class   Address  Height  Weight  Math Physics
Gender School                                              
F      S_1      C_1  street_2     192      73  32.5      B+
       S_1      C_1  street_2     167      81  80.4      B-

(('F', 'S_2'), 66.42727272727274) 66.42727272727274


Class   Address  Height  Weight  Math Physics
Gender School                                              
F      S_2      C_1  street_6     161      61  50.6      B+
       S_2      C_1  street_5     159      97  72.2      B+

(('M', 'S_1'), 63.34285714285714) 63.34285714285714


Class   Address  Height  Weight  Math Physics
Gender School                                              
M      S_1      C_1  street_1     173      63  34.0      A+
       S_1      C_1  street_2     186      82  87.2      B+

(('M', 'S_2'), 51.155555555555544) 51.155555555555544


Class   Address  Height  Weight  Math Physics
Gender School                                              
M      S_2      C_1  street_7     174      84  83.3       C
       S_2      C_1  street_4     157      61  52.5      B-

#### （d）groupby的[]操作
#### 可以用[]选出groupby对象的某个或者某几个列，上面的均分比较可以如下简洁地写出：

In [145]:
df.groupby(['Gender','School'])['Math'].mean()>=60

Gender  School
F       S_1        True
        S_2        True
M       S_1        True
        S_2       False
Name: Math, dtype: bool

In [171]:
# 相当于将['Math'].mean()>=60 传递给子df 然后再把结果combine
# combine的时候会把分组列设置为索引,而不是如下这样先设置层次索引再取mean再combine--这种情况下取mean会丢掉索引.
#df_=pd.DataFrame()
groupby2=df.groupby(['Gender','School'])
for groups in groupby2:
    display(groups[1].set_index(['Gender','School'])['Math'])#.mean()>=60)
    #df_=pd.concat([df_, groups[1]['Math'].mean()>=60])
#df_

Gender  School
F       S_1       32.5
        S_1       80.4
        S_1       84.8
        S_1       63.5
        S_1       33.8
        S_1       68.4
        S_1       87.7
        S_1       61.7
Name: Math, dtype: float64

Gender  School
F       S_2       50.6
        S_2       72.2
        S_2       68.5
        S_2       85.4
        S_2       72.3
        S_2       65.9
        S_2       95.5
        S_2       45.3
        S_2       59.7
        S_2       67.7
        S_2       47.6
Name: Math, dtype: float64

Gender  School
M       S_1       34.0
        S_1       87.2
        S_1       97.0
        S_1       58.8
        S_1       31.5
        S_1       49.7
        S_1       85.2
Name: Math, dtype: float64

Gender  School
M       S_2       83.3
        S_2       52.5
        S_2       34.2
        S_2       39.1
        S_2       73.8
        S_2       47.2
        S_2       32.7
        S_2       48.9
        S_2       48.7
Name: Math, dtype: float64

In [ ]:
# 由于分组结果是可以迭代的,实际上是可以写个循环来实现与 df.groupby(['Gender','School'])['Math'].mean()>=60  相同的效果


In [156]:
pd.concat??

Signature:
pd.concat(
    objs: Union[Iterable[Union[ForwardRef('DataFrame'), ForwardRef('Series')]], Mapping[Union[Hashable, NoneType], Union[ForwardRef('DataFrame'), ForwardRef('Series')]]],
    axis=0,
    join='outer',
    ignore_index: bool = False,
    keys=None,
    levels=None,
    names=None,
    verify_integrity: bool = False,
    sort: bool = False,
    copy: bool = True,
) -> Union[ForwardRef('DataFrame'), ForwardRef('Series')]
Source:   
def concat(
    objs: Union[
        Iterable[FrameOrSeriesUnion], Mapping[Optional[Hashable], FrameOrSeriesUnion]
    ],
    axis=0,
    join="outer",
    ignore_index: bool = False,
    keys=None,
    levels=None,
    names=None,
    verify_integrity: bool = False,
    sort: bool = False,
    copy: bool = True,
) -> FrameOrSeriesUnion:
    """
    Concatenate pandas objects along a particular axis with optional set logic
    along the other axes.

    Can also add a layer of hierarchical indexing on the concatenation axis,
    which may 

#### 用列表可选出多个属性列：

In [172]:
df.groupby(['Gender','School'])[['Math','Height']].mean()
# 注意[]内的应该是个list,因此显示效果应该有两层[]

Math      Height
Gender School                       
F      S_1     64.100000  173.125000
       S_2     66.427273  173.727273
M      S_1     63.342857  178.714286
       S_2     51.155556  172.000000

#### （e）连续型变量分组
#### 例如利用cut函数对数学成绩分组：

In [178]:
bins = [0,40,60,80,90,100]
cuts = pd.cut(df['Math'],bins=bins) #可选label添加自定义标签
df.groupby(cuts)['Math'].count()

Math
(0, 40]       7
(40, 60]     10
(60, 80]      9
(80, 90]      7
(90, 100]     2
Name: Math, dtype: int64

In [173]:
pd.cut??

Signature:
pd.cut(
    x,
    bins,
    right: bool = True,
    labels=None,
    retbins: bool = False,
    precision: int = 3,
    include_lowest: bool = False,
    duplicates: str = 'raise',
)
Source:   
def cut(
    x,
    bins,
    right: bool = True,
    labels=None,
    retbins: bool = False,
    precision: int = 3,
    include_lowest: bool = False,
    duplicates: str = "raise",
):
    """
    Bin values into discrete intervals.

    Use `cut` when you need to segment and sort data values into bins. This
    function is also useful for going from a continuous variable to a
    categorical variable. For example, `cut` could convert ages to groups of
    age ranges. Supports binning into an equal number of bins, or a
    pre-specified array of bins.

    Parameters
    ----------
    x : array-like
        The input array to be binned. Must be 1-dimensional.
    bins : int, sequence of scalars, or IntervalIndex
        The criteria to bin by.

        * int : Defines the number of

In [174]:
pd.qcut??

Signature:
pd.qcut(
    x,
    q,
    labels=None,
    retbins: bool = False,
    precision: int = 3,
    duplicates: str = 'raise',
)
Source:   
def qcut(
    x,
    q,
    labels=None,
    retbins: bool = False,
    precision: int = 3,
    duplicates: str = "raise",
):
    """
    Quantile-based discretization function.

    Discretize variable into equal-sized buckets based on rank or based
    on sample quantiles. For example 1000 values for 10 quantiles would
    produce a Categorical object indicating quantile membership for each data point.

    Parameters
    ----------
    x : 1d ndarray or Series
    q : int or list-like of int
        Number of quantiles. 10 for deciles, 4 for quartiles, etc. Alternately
        array of quantiles, e.g. [0, .25, .5, .75, 1.] for quartiles.
    labels : array or False, default None
        Used as labels for the resulting bins. Must be of the same length as
        the resulting bins. If False, return only integer indicators of the
        bi

In [179]:
# 按成绩高低分为五个组,每个组包含的个数相等
qcuts=pd.qcut(df.Math,5)
df.groupby(qcuts)['Math'].describe()

,count,mean,std,min,25%,50%,75%,max
Math,,,,,,,,
"(31.499, 44.06]",7.0,33.971429,2.458803,31.5,32.60,33.8,34.1,39.1
"(44.06, 51.74]",7.0,48.285714,1.754450,45.3,47.40,48.7,49.3,50.6
"(51.74, 67.98]",7.0,61.400000,5.054701,52.5,59.25,61.7,64.7,67.7
"(67.98, 83.6]",7.0,74.128571,5.702547,68.4,70.35,72.3,77.1,83.3
"(83.6, 97.0]",7.0,88.971429,5.102520,84.8,85.30,87.2,91.6,97.0


## 三、聚合、过滤和变换

### 1. 聚合（Aggregation）

#### （a）常用聚合函数
#### 所谓聚合就是把一堆数，变成一个标量，因此mean/sum/size/count/std/var/sem/describe/first/last/nth/min/max都是聚合函数
#### 为了熟悉操作，不妨验证标准误sem函数，它的计算公式是：$\frac{组内标准差}{\sqrt{组容量}}$，下面进行验证：

In [180]:
group_m = grouped_single['Math']
group_m.std().values/np.sqrt(group_m.count().values)== group_m.sem().values

array([ True,  True])

In [184]:
group_m.nth??
# Take the nth row from each group if n is an int, or a subset of rows if n is a list of ints.

Signature: group_m.nth(n: Union[int, List[int]], dropna: Union[str, NoneType] = None) -> pandas.core.frame.DataFrame
Docstring:
Take the nth row from each group if n is an int, or a subset of rows
if n is a list of ints.

If dropna, will take the nth non-null row, dropna is either
'all' or 'any'; this is equivalent to calling dropna(how=dropna)
before the groupby.

Parameters
----------
n : int or list of ints
    A single nth value for the row or a list of nth values.
dropna : None or str, optional
    Apply the specified dropna operation before counting which row is
    the nth row. Needs to be None, 'any' or 'all'.

Returns
-------
Series or DataFrame
    N-th value within each group.

See Also
--------
Series.groupby
DataFrame.groupby

Examples
--------

>>> df = pd.DataFrame({'A': [1, 1, 2, 1, 2],
...                    'B': [np.nan, 2, 3, 4, 5]}, columns=['A', 'B'])
>>> g = df.groupby('A')
>>> g.nth(0)
     B
A
1  NaN
2  3.0
>>> g.nth(1)
     B
A
1  2.0
2  5.0
>>> g.nth(-1)
     

#### （b）同时使用多个聚合函数

In [185]:
group_m.agg(['sum','mean','std'])
# 传入多个内置聚合函数, 函数名需要加引号

,sum,mean,std
School,,,
S_1,956.2,63.746667,23.077474
S_2,1191.1,59.555000,17.589305


#### 利用元组进行重命名

In [186]:
group_m.agg([('rename_sum','sum'),('rename_mean','mean')])
# 列名放在元组的第一个位置, 需要应用的内置函数的函数名放在第二个位置--有些不太一致
# select sum(Math) as rename_sum, mean(Math) as rename_mean from df group by School

,rename_sum,rename_mean
School,,
S_1,956.2,63.746667
S_2,1191.1,59.555000


In [200]:
group_m.agg([('rename_sum','sum'),('rename_range',lambda x: x.max()-x.min())])
# 传入lambda函数也是可以的

,rename_sum,rename_range
School,,
S_1,956.2,65.5
S_2,1191.1,62.8


In [203]:
def range_1(x):
    return x.max()-x.min()
group_m.agg([('rename_sum','sum'),('rename_range_1',range_1)])
# 传入自定义函数也是可以的--但这时函数名不能加引号

,rename_sum,rename_range_1
School,,
S_1,956.2,65.5
S_2,1191.1,62.8


In [204]:
group_m.agg??
# 类似于apply, 但必须传入聚合函数,或聚合函数做为变量构成的复合函数

Signature: group_m.agg(func=None, *args, **kwargs)
Docstring:
Aggregate using one or more operations over the specified axis.

Parameters
----------
func : function, str, list or dict
    Function to use for aggregating the data. If a function, must either
    work when passed a Series or when passed to Series.apply.

    Accepted combinations are:

    - function
    - string function name
    - list of functions and/or function names, e.g. ``[np.sum, 'mean']``
    - dict of axis labels -> functions, function names or list of such.

*args
    Positional arguments to pass to `func`.
**kwargs
    Keyword arguments to pass to `func`.

Returns
-------
scalar, Series or DataFrame

    The return can be:

    * scalar : when Series.agg is called with single function
    * Series : when DataFrame.agg is called with a single function
    * DataFrame : when DataFrame.agg is called with several functions

    Return scalar, Series or DataFrame.

See Also
--------
pandas.Series.groupby.apply
pan

#### 指定哪些函数作用哪些列

In [205]:
grouped_mul.agg({'Math':['mean','max'],'Height':'var'})
# 这时是不支持对计算得到的列进行重命名的.

Math       Height
               mean   max    var
School Class                    
S_1    C_1    63.78  87.2  183.3
       C_2    64.30  97.0  132.8
       C_3    63.16  87.7  179.2
S_2    C_1    58.56  83.3   54.7
       C_2    62.80  85.4  256.0
       C_3    63.06  95.5  205.7
       C_4    53.80  67.7  300.2

#### （c）使用自定义函数

In [206]:
grouped_single['Math'].agg(lambda x:print(x.head(),'间隔'))
#可以发现，agg函数的传入是分组逐列进行的，有了这个特性就可以做许多事情

1101    34.0
1102    32.5
1103    87.2
1104    80.4
1105    84.8
Name: Math, dtype: float64 间隔
2101    83.3
2102    50.6
2103    52.5
2104    72.2
2105    34.2
Name: Math, dtype: float64 间隔


School
S_1    None
S_2    None
Name: Math, dtype: object

#### 官方没有提供极差计算的函数，但通过agg可以容易地实现组内极差计算

In [207]:
grouped_single['Math'].agg(lambda x:x.max()-x.min())

School
S_1    65.5
S_2    62.8
Name: Math, dtype: float64

#### （d）利用NamedAgg函数进行多个聚合
#### 注意：不支持lambda函数，但是可以使用外置的def函数

In [209]:
# 当需要多不同列应用不同函数,并对得到的列重命名时, 使用NamedAgg函数
def R1(x):
    return x.max()-x.min()
def R2(x):
    return x.max()-x.median()
grouped_single['Math'].agg(min_score1=pd.NamedAgg(column='col1', aggfunc=R1),
                           max_score1=pd.NamedAgg(column='col2', aggfunc='max'),
                           range_score2=pd.NamedAgg(column='col3', aggfunc=R2))#.head()

,min_score1,max_score1,range_score2
School,,,
S_1,65.5,97.0,33.5
S_2,62.8,95.5,39.4


In [208]:
pd.NamedAgg??

Init signature: pd.NamedAgg(column, aggfunc)
Docstring:      NamedAgg(column, aggfunc)
File:           d:\programdata\anaconda3\lib\site-packages\pandas\core\groupby\generic.py
Type:           type
Subclasses:     


#### （e）带参数的聚合函数
#### 判断是否组内数学分数至少有一个值在50-52之间：

In [210]:
def f(s,low,high):
    return s.between(low,high).max()
grouped_single['Math'].agg(f,50,52)
# 注意agg的参数: 第一个参数是函数名, 第二三个参数是f的第二三个参数, f的第一个参数由agg前的变量提供.
# 类似于偏函数

School
S_1    False
S_2     True
Name: Math, dtype: bool

#### 如果需要使用多个函数，并且其中至少有一个带参数，则使用wrap技巧：

In [211]:
# 关于 wrap, 参考Python函数中的装饰器.
# 更多内容可参考Python函数式编程的相关资料
def f_test(s,low,high):
    return s.between(low,high).max()
def agg_f(f_mul,name,*args,**kwargs):
    def wrapper(x):
        return f_mul(x,*args,**kwargs)
    wrapper.__name__ = name # 这一步骤最关键, 将装饰器的函数名重新定义为原来的函数名,否则列名就会变成wrapper
    return wrapper
new_f = agg_f(f_test,'at_least_one_in_50_52',50,52)
grouped_single['Math'].agg([new_f,'mean']).head()

,at_least_one_in_50_52,mean
School,,
S_1,False,63.746667
S_2,True,59.555000


In [212]:
# 关于 wrap, 参考Python函数中的装饰器.
# 更多内容可参考Python函数式编程的相关资料
def f_test(s,low,high):
    return s.between(low,high).max()
def agg_f(f_mul,name,*args,**kwargs):
    def wrapper(x):
        return f_mul(x,*args,**kwargs)
    #wrapper.__name__ = name # 这一步骤最关键, 将装饰器的函数名重新定义为原来的函数名,否则列名就会变成wrapper #注释掉后,列名就变成了wrapper
    return wrapper
new_f = agg_f(f_test,'at_least_one_in_50_52',50,52)
grouped_single['Math'].agg([new_f,'mean']).head()

,wrapper,mean
School,,
S_1,False,63.746667
S_2,True,59.555000


### 2. 过滤（Filteration）
#### filter函数是用来筛选某些组的（务必记住结果是组的全体），因此传入的值应当是布尔标量

In [289]:
grouped_single.filter(lambda x:(x['Math']>=32).all()).T
# 由于传入了.all() 因此返回的是组内的全体都满足(['Math']>=32)条件的组--实际上也就是School=S_2的那一组,一共有20行.

ID,2101,2102,2103,2104,2105,2201,2202,2203,2204,2205,2301,2302,2303,2304,2305,2401,2402,2403,2404,2405
Class,C_1,C_1,C_1,C_1,C_1,C_2,C_2,C_2,C_2,C_2,C_3,C_3,C_3,C_3,C_3,C_4,C_4,C_4,C_4,C_4
Gender,M,F,M,F,M,M,F,M,M,F,F,M,F,F,M,F,M,F,F,F
Address,street_7,street_6,street_4,street_5,street_4,street_5,street_7,street_4,street_1,street_7,street_4,street_5,street_7,street_6,street_4,street_2,street_7,street_6,street_2,street_6
Height,174,161,157,159,170,193,194,155,175,183,157,171,190,164,187,192,166,158,160,193
Weight,84,61,61,97,81,100,77,91,74,76,78,88,99,81,73,62,82,60,84,54
Math,83.3,50.6,52.5,72.2,34.2,39.1,68.5,73.8,47.2,85.4,72.3,32.7,65.9,95.5,48.9,45.3,48.7,59.7,67.7,47.6
Physics,C,B+,B-,B+,A,B,B+,A+,B-,B,B+,A,C,A-,B,A,B,B+,B,B


In [226]:
# 如果将条件改为"分组内的所有行的数学成绩都大于33",怎会返回空df
grouped_single.filter(lambda x:(x['Math']>33).all())

,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,


In [288]:
# 将 .all()换成 .any(), 则选出那些"组内存在数学成绩大于指定值"的组
# 由于S_1分组中也有数学成绩大于32的, 因此也会被返回
grouped_single.filter(lambda x:(x['Math']>=32).any()).T#.head()
# 注意观察返回的结果, 可以发现, 并没有显示School列(不论是作为索引还是列)

ID,1101,1102,1103,1104,1105,1201,1202,1203,1204,1205,...,2301,2302,2303,2304,2305,2401,2402,2403,2404,2405
Class,C_1,C_1,C_1,C_1,C_1,C_2,C_2,C_2,C_2,C_2,...,C_3,C_3,C_3,C_3,C_3,C_4,C_4,C_4,C_4,C_4
Gender,M,F,M,F,F,M,F,M,F,F,...,F,M,F,F,M,F,M,F,F,F
Address,street_1,street_2,street_2,street_2,street_4,street_5,street_4,street_6,street_5,street_6,...,street_4,street_5,street_7,street_6,street_4,street_2,street_7,street_6,street_2,street_6
Height,173,192,186,167,159,188,176,160,162,167,...,157,171,190,164,187,192,166,158,160,193
Weight,63,73,82,81,64,68,94,53,63,63,...,78,88,99,81,73,62,82,60,84,54
Math,34,32.5,87.2,80.4,84.8,97,63.5,58.8,33.8,68.4,...,72.3,32.7,65.9,95.5,48.9,45.3,48.7,59.7,67.7,47.6
Physics,A+,B+,B+,B-,B+,A-,B-,A+,B,B-,...,B+,A,C,A-,B,A,B,B+,B,B


In [259]:
# 来看一下返回的行的School
df.School[grouped_single.filter(lambda x:(x['Math']>32).all()).index]
# 如果分组列事前并未排序,且唯一值有多个, 并且只有个别组满足筛选条件, 那么就会发现返回的结果已经按照分组后的顺序重新排列了---并没有,还是原始df中的顺序

ID
2101    S_2
2102    S_2
2103    S_2
2104    S_2
2105    S_2
2201    S_2
2202    S_2
2203    S_2
2204    S_2
2205    S_2
2301    S_2
2302    S_2
2303    S_2
2304    S_2
2305    S_2
2401    S_2
2402    S_2
2403    S_2
2404    S_2
2405    S_2
Name: School, dtype: object

In [253]:
df.School[grouped_single.filter(lambda x:(x['Math']<32).any()).index]

ID
1101    S_1
1102    S_1
1103    S_1
1104    S_1
1105    S_1
1201    S_1
1202    S_1
1203    S_1
1204    S_1
1205    S_1
1301    S_1
1302    S_1
1303    S_1
1304    S_1
1305    S_1
Name: School, dtype: object

In [246]:
len(grouped_single),len(df),len(grouped_single.filter(lambda x:(x['Math']>32).all())),

(2, 35, 20)

In [249]:
len(grouped_single.filter(lambda x:(x['Math']>32).all())), len(grouped_single.filter(lambda x:(x['Math']>32).any())), 
# .all() 筛选出那些"组内所有行的数学成绩大于32"的组, .any() 筛选出那些"组内存在某一行的数学成绩大于32"的组

(20, 35)

In [287]:
# 重新分组--按照具有多个取值且未按照该列排序的列 Class 进行分组,然后用数学成绩来筛选
df.groupby('Class').filter(lambda x:(x['Math']>32).all()).T

ID,1101,1102,1103,1104,1105,1201,1202,1203,1204,1205,...,2201,2202,2203,2204,2205,2401,2402,2403,2404,2405
School,S_1,S_1,S_1,S_1,S_1,S_1,S_1,S_1,S_1,S_1,...,S_2,S_2,S_2,S_2,S_2,S_2,S_2,S_2,S_2,S_2
Class,C_1,C_1,C_1,C_1,C_1,C_2,C_2,C_2,C_2,C_2,...,C_2,C_2,C_2,C_2,C_2,C_4,C_4,C_4,C_4,C_4
Gender,M,F,M,F,F,M,F,M,F,F,...,M,F,M,M,F,F,M,F,F,F
Address,street_1,street_2,street_2,street_2,street_4,street_5,street_4,street_6,street_5,street_6,...,street_5,street_7,street_4,street_1,street_7,street_2,street_7,street_6,street_2,street_6
Height,173,192,186,167,159,188,176,160,162,167,...,193,194,155,175,183,192,166,158,160,193
Weight,63,73,82,81,64,68,94,53,63,63,...,100,77,91,74,76,62,82,60,84,54
Math,34,32.5,87.2,80.4,84.8,97,63.5,58.8,33.8,68.4,...,39.1,68.5,73.8,47.2,85.4,45.3,48.7,59.7,67.7,47.6
Physics,A+,B+,B+,B-,B+,A-,B-,A+,B,B-,...,B,B+,A+,B-,B,A,B,B+,B,B


In [274]:
len(df.groupby('Class').filter(lambda x:(x['Math']>32).all()))

25

In [276]:
# 被筛选掉的 Class=C_3的分组
df.groupby('Class').filter(lambda x:(x['Math']<32).any())

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1301,S_1,C_3,M,street_4,161,68,31.5,B+
1302,S_1,C_3,F,street_1,175,57,87.7,A-
1303,S_1,C_3,M,street_7,188,82,49.7,B
1304,S_1,C_3,M,street_2,195,70,85.2,A
1305,S_1,C_3,F,street_5,187,69,61.7,B-
2301,S_2,C_3,F,street_4,157,78,72.3,B+
2302,S_2,C_3,M,street_5,171,88,32.7,A
2303,S_2,C_3,F,street_7,190,99,65.9,C
2304,S_2,C_3,F,street_6,164,81,95.5,A-


In [277]:
grouped_single.filter??
# Return a copy of a DataFrame excluding elements from groups that do not satisfy the boolean criterion specified by func.

Signature: grouped_single.filter(func, dropna=True, *args, **kwargs)
Source:   
    def filter(self, func, dropna=True, *args, **kwargs):
        """
        Return a copy of a DataFrame excluding elements from groups that
        do not satisfy the boolean criterion specified by func.

        Parameters
        ----------
        f : function
            Function to apply to each subframe. Should return True or False.
        dropna : Drop groups that do not pass the filter. True by default;
            If False, groups that evaluate False are filled with NaNs.

        Returns
        -------
        filtered : DataFrame

        Notes
        -----
        Each subframe is endowed the attribute 'name' in case you need to know
        which group you are working on.

        Examples
        --------
        >>> df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
        ...                           'foo', 'bar'],
        ...                    'B' : [1, 2, 3, 4, 5, 6],
        .

### 3. 变换（Transformation）
#### （a）传入对象
#### transform函数中传入的对象是组内的列，并且返回值需要与列长完全一致

In [278]:
grouped_single[['Math','Height']].transform(lambda x:x-x.min()).head()

,Math,Height
ID,,
1101,2.5,14
1102,1.0,33
1103,55.7,27
1104,48.9,8
1105,53.3,0


In [281]:
grouped_single.transform??
#Call function producing a like-indexed DataFrame on each group and return a DataFrame having the same indexes as the original object filled with the transformed values

Signature: grouped_single.transform(func, *args, **kwargs)
Docstring:
Call function producing a like-indexed DataFrame on each group and
return a DataFrame having the same indexes as the original object
filled with the transformed values

Parameters
----------
f : function
    Function to apply to each group

Returns
-------
DataFrame

See Also
--------
aggregate, transform

Notes
-----
Each group is endowed the attribute 'name' in case you need to know
which group you are working on.

The current implementation imposes three requirements on f:

* f must return a value that either has the same shape as the input
  subframe or can be broadcast to the shape of the input subframe.
  For example, if `f` returns a scalar it will be broadcast to have the
  same shape as the input subframe.
* if this is a DataFrame, f must support application column-by-column
  in the subframe. If f also supports application to the entire subframe,
  then a fast path is used starting from the second chunk.
* 

#### 如果返回了标量值，那么组内的所有元素会被广播为这个值

In [284]:
grouped_single[['Math','Height']].transform(lambda x:x.mean()).head()

,Math,Height
ID,,
1101,63.746667,175.733333
1102,63.746667,175.733333
1103,63.746667,175.733333
1104,63.746667,175.733333
1105,63.746667,175.733333


In [286]:
grouped_single.transform(lambda x:x.mean()).T#.head()
# 非数值列未返回--类似于describe方法

ID,1101,1102,1103,1104,1105,1201,1202,1203,1204,1205,...,2301,2302,2303,2304,2305,2401,2402,2403,2404,2405
Height,175.733333,175.733333,175.733333,175.733333,175.733333,175.733333,175.733333,175.733333,175.733333,175.733333,...,172.950,172.950,172.950,172.950,172.950,172.950,172.950,172.950,172.950,172.950
Weight,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,...,78.150,78.150,78.150,78.150,78.150,78.150,78.150,78.150,78.150,78.150
Math,63.746667,63.746667,63.746667,63.746667,63.746667,63.746667,63.746667,63.746667,63.746667,63.746667,...,59.555,59.555,59.555,59.555,59.555,59.555,59.555,59.555,59.555,59.555


In [291]:
df.join(grouped_single.transform(lambda x:x.mean()),rsuffix='_grouped')
# 实现了sql中的窗口函数的效果

,School,Class,Gender,Address,Height,Weight,Math,Physics,Height_grouped,Weight_grouped,Math_grouped
ID,,,,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+,175.733333,70.00,63.746667
1102,S_1,C_1,F,street_2,192,73,32.5,B+,175.733333,70.00,63.746667
1103,S_1,C_1,M,street_2,186,82,87.2,B+,175.733333,70.00,63.746667
1104,S_1,C_1,F,street_2,167,81,80.4,B-,175.733333,70.00,63.746667
1105,S_1,C_1,F,street_4,159,64,84.8,B+,175.733333,70.00,63.746667
1201,S_1,C_2,M,street_5,188,68,97.0,A-,175.733333,70.00,63.746667
1202,S_1,C_2,F,street_4,176,94,63.5,B-,175.733333,70.00,63.746667
1203,S_1,C_2,M,street_6,160,53,58.8,A+,175.733333,70.00,63.746667
1204,S_1,C_2,F,street_5,162,63,33.8,B,175.733333,70.00,63.746667


In [312]:
df.join(df.groupby(df.Class).transform(lambda x:x.mean()),rsuffix='_grouped')
# 确实是实现了窗口函数的效果
# 以oracle为例 等价于: 
# select df.*,A.* from df left outer join 
# (select avg(Height) as Height_grouped, avg(Weight) as Weight_grouped, avg(Math) as Math_grouped from df group by Class) A on df.index=A.index

,School,Class,Gender,Address,Height,Weight,Math,Physics,Height_grouped,Weight_grouped,Math_grouped
ID,,,,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+,169.8,74.7,61.17
1102,S_1,C_1,F,street_2,192,73,32.5,B+,169.8,74.7,61.17
1103,S_1,C_1,M,street_2,186,82,87.2,B+,169.8,74.7,61.17
1104,S_1,C_1,F,street_2,167,81,80.4,B-,169.8,74.7,61.17
1105,S_1,C_1,F,street_4,159,64,84.8,B+,169.8,74.7,61.17
1201,S_1,C_2,M,street_5,188,68,97.0,A-,175.3,75.9,63.55
1202,S_1,C_2,F,street_4,176,94,63.5,B-,175.3,75.9,63.55
1203,S_1,C_2,M,street_6,160,53,58.8,A+,175.3,75.9,63.55
1204,S_1,C_2,F,street_5,162,63,33.8,B,175.3,75.9,63.55


#### （b）利用变换方法进行组内标准化

In [313]:
grouped_single[['Math','Height']].transform(lambda x:(x-x.mean())/x.std()).head()
# 标准化就是将数据转化为均值为0标准差为1的新数据, 本质上是将定距变量转换为定比变量
# 消除了不同量纲或不同方差,均值带来的影响

,Math,Height
ID,,
1101,-1.288991,-0.214991
1102,-1.353990,1.279460
1103,1.016287,0.807528
1104,0.721627,-0.686923
1105,0.912289,-1.316166


In [ ]:
# transform和只能进行聚合的agg显然不同, 同时也不同于只能对分组进行筛选的filter
# transform可以通过传入各种函数, 来对分组的结果按组进行组内计算,并且不进行聚合
# 在需要对数据分组设置不同的处理方案时特别有用--例如,用每组的均值填充组内的缺失值

#### （c）利用变换方法进行组内缺失值的均值填充

In [318]:
# 生成具有缺失值的数据:数据的初始化
df_nan = df[['School','Math','Height','Weight']].copy().reset_index()
df_nan.loc[np.random.randint(0,df.shape[0],25),['Math','Height','Weight']]=np.nan # 数学成绩引入了25个缺失值
df_nan.head()

,ID,School,Math,Height,Weight
0,1101,S_1,34.0,173.0,63.0
1,1102,S_1,NaN,NaN,NaN
2,1103,S_1,NaN,NaN,NaN
3,1104,S_1,NaN,NaN,NaN
4,1105,S_1,NaN,NaN,NaN


In [326]:
df_nan.groupby('School').transform(lambda x: x.fillna(x.mean())).join(df.reset_index()['School']).T#.head()
# 对按School分组的每组数据, 将组内每一列的缺失值用该组该列的均值进行替换.

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
ID,1101,1102,1103,1104,1105,1201,1202,1203,1204,1205,...,2301,2302,2303,2304,2305,2401,2402,2403,2404,2405
Math,34,61.375,61.375,61.375,61.375,97,63.5,61.375,61.375,68.4,...,61.625,61.625,61.625,61.625,48.9,61.625,61.625,61.625,67.7,61.625
Height,173,179.375,179.375,179.375,179.375,188,176,179.375,179.375,167,...,172,172,172,172,187,172,172,172,160,172
Weight,63,72.125,72.125,72.125,72.125,68,94,72.125,72.125,63,...,75.375,75.375,75.375,75.375,73,75.375,75.375,75.375,84,75.375
School,S_1,S_1,S_1,S_1,S_1,S_1,S_1,S_1,S_1,S_1,...,S_2,S_2,S_2,S_2,S_2,S_2,S_2,S_2,S_2,S_2


In [327]:
# 检查分组 S_1 均值等统计量
df_nan[df_nan['School']=='S_1'].describe()

,ID,Math,Height,Weight
count,15.000000,8.000000,8.000000,8.000000
mean,1203.000000,61.375000,179.375000,72.125000
std,84.528102,22.894026,11.915626,10.629306
min,1101.000000,31.500000,161.000000,63.000000
25%,1104.500000,45.775000,171.500000,66.750000
50%,1203.000000,62.600000,181.500000,68.500000
75%,1301.500000,72.600000,188.000000,73.000000
max,1305.000000,97.000000,195.000000,94.000000


In [328]:
# 检查分组 S_1 填充后的均值
df_nan.groupby('School').transform(lambda x: x.fillna(x.mean())).join(df.reset_index()['School'])[df_nan['School']=='S_1'].describe()

,ID,Math,Height,Weight
count,15.000000,15.000000,15.00000,15.000000
mean,1203.000000,61.375000,179.37500,72.125000
std,84.528102,16.188521,8.42562,7.516054
min,1101.000000,31.500000,161.00000,63.000000
25%,1104.500000,61.375000,177.68750,68.500000
50%,1203.000000,61.375000,179.37500,72.125000
75%,1301.500000,62.600000,183.18750,72.125000
max,1305.000000,97.000000,195.00000,94.000000


In [329]:
# 检查分组 S_2 均值等统计量
df_nan[df_nan['School']=='S_2'].describe()

,ID,Math,Height,Weight
count,20.000000,8.000000,8.000000,8.00000
mean,2253.000000,61.625000,172.000000,75.37500
std,114.717043,13.815493,14.628739,11.74658
min,2101.000000,47.200000,157.000000,61.00000
25%,2177.000000,50.175000,159.750000,70.00000
50%,2253.000000,60.100000,168.000000,75.00000
75%,2329.000000,69.425000,184.000000,78.75000
max,2405.000000,85.400000,194.000000,97.00000


In [330]:
# 检查分组 S_2 填充后的均值
df_nan.groupby('School').transform(lambda x: x.fillna(x.mean())).join(df.reset_index()['School'])[df_nan['School']=='S_2'].describe()

,ID,Math,Height,Weight
count,20.000000,20.000000,20.000000,20.000000
mean,2253.000000,61.625000,172.000000,75.375000
std,114.717043,8.385686,8.879308,7.129904
min,2101.000000,47.200000,157.000000,61.000000
25%,2177.000000,61.625000,172.000000,75.375000
50%,2253.000000,61.625000,172.000000,75.375000
75%,2329.000000,61.625000,172.000000,75.375000
max,2405.000000,85.400000,194.000000,97.000000


## 四、apply函数

### 1. apply函数的灵活性
#### 可能在所有的分组函数中，apply是应用最为广泛的，这得益于它的灵活性：
#### 对于传入值而言，从下面的打印内容可以看到是以分组的表传入apply中：

In [331]:
df.groupby('School').apply(lambda x:print(x.head(1)))

     School Class Gender   Address  Height  Weight  Math Physics
ID                                                              
1101    S_1   C_1      M  street_1     173      63  34.0      A+
     School Class Gender   Address  Height  Weight  Math Physics
ID                                                              
2101    S_2   C_1      M  street_7     174      84  83.3       C


""


#### apply函数的灵活性很大程度来源于其返回值的多样性：

#### ① 标量返回值

In [332]:
df[['School','Math','Height']].groupby('School').apply(lambda x:x.max())

,School,Math,Height
School,,,
S_1,S_1,97.0,195
S_2,S_2,95.5,194


In [333]:
# 实现了agg的聚合功能
df[['School','Math','Height']].groupby('School').agg(lambda x:x.max())

,Math,Height
School,,
S_1,97.0,195
S_2,95.5,194


#### ② 列表返回值

In [334]:
df[['School','Math','Height']].groupby('School').apply(lambda x:x-x.min())#.head()
# 实现了transfom的分组区别计算的功能
# 比较不完美的是, School 列不见了

,Math,Height
ID,,
1101,2.5,14.0
1102,1.0,33.0
1103,55.7,27.0
1104,48.9,8.0
1105,53.3,0.0
1201,65.5,29.0
1202,32.0,17.0
1203,27.3,1.0
1204,2.3,3.0


#### ③ 数据框返回值

In [340]:
df[['School','Math','Height']].groupby('School')\
    .apply(lambda x:pd.DataFrame({'col1':x['Math']-x['Math'].max(),
                                  'col2':x['Math']-x['Math'].min(),
                                  'col3':x['Height']-x['Height'].max(),
                                  'col4':x['Height']-x['Height'].min()}))#.head()
# 相当于在沿用原始df的index的基础上, 使用原始df的分组, 构造了新的列并用新的列构造了一个df--这在利用基础数据进行报表统计时非常方便

,col1,col2,col3,col4
ID,,,,
1101,-63.0,2.5,-22,14
1102,-64.5,1.0,-3,33
1103,-9.8,55.7,-9,27
1104,-16.6,48.9,-28,8
1105,-12.2,53.3,-36,0
1201,0.0,65.5,-7,29
1202,-33.5,32.0,-19,17
1203,-38.2,27.3,-35,1
1204,-63.2,2.3,-33,3


In [343]:
df[['School','Math','Height']].groupby('School')\
    .apply(lambda x:pd.DataFrame({'School':x['School'], # 将原始df的School列显示出来
                                  'col1':x['Math']-x['Math'].max(),
                                  'col2':x['Math']-x['Math'].min(),
                                  'col3':x['Height']-x['Height'].max(),
                                  'col4':x['Height']-x['Height'].min()}))#.head()
# 相当于在沿用原始df的index的基础上, 使用原始df的分组, 构造了新的列并用新的列构造了一个df--这在利用基础数据进行报表统计时非常方便

,School,col1,col2,col3,col4
ID,,,,,
1101,S_1,-63.0,2.5,-22,14
1102,S_1,-64.5,1.0,-3,33
1103,S_1,-9.8,55.7,-9,27
1104,S_1,-16.6,48.9,-28,8
1105,S_1,-12.2,53.3,-36,0
1201,S_1,0.0,65.5,-7,29
1202,S_1,-33.5,32.0,-19,17
1203,S_1,-38.2,27.3,-35,1
1204,S_1,-63.2,2.3,-33,3


### 2. 用apply同时统计多个指标
#### 此处可以借助OrderedDict工具进行快捷的统计：

In [344]:
from collections import OrderedDict
OrderedDict??

Init signature: OrderedDict(self, /, *args, **kwargs)
Source:        
class OrderedDict(dict):
    'Dictionary that remembers insertion order'
    # An inherited dict maps keys to values.
    # The inherited dict provides __getitem__, __len__, __contains__, and get.
    # The remaining methods are order-aware.
    # Big-O running times for all methods are the same as regular dictionaries.

    # The internal self.__map dict maps keys to links in a doubly linked list.
    # The circular doubly linked list starts and ends with a sentinel element.
    # The sentinel element never gets deleted (this simplifies the algorithm).
    # The sentinel is in self.__hardroot with a weakref proxy in self.__root.
    # The prev links are weakref proxies (to prevent circular references).
    # Individual links are kept alive by the hard reference in self.__map.
    # Those hard references disappear when a key is deleted from an OrderedDict.

    def __init__(*args, **kwds):
        '''Initialize an or

In [347]:
data = OrderedDict()
data

OrderedDict()

In [348]:
def f(df):
    data = OrderedDict()
    data['M_sum'] = df['Math'].sum()
    data['W_var'] = df['Weight'].var()
    data['H_mean'] = df['Height'].mean()
    return pd.Series(data)

In [349]:
grouped_single.apply(f)

,M_sum,W_var,H_mean
School,,,
S_1,956.2,117.428571,175.733333
S_2,1191.1,181.081579,172.950000


In [350]:
df.groupby('Class').apply(f)

,M_sum,W_var,H_mean
Class,,,
C_1,611.7,149.566667,169.8
C_2,635.5,228.988889,175.3
C_3,631.1,139.388889,177.5
C_4,269.0,186.800000,173.8


## 五、问题与练习

### 1. 问题

#### 【问题一】 什么是fillna的前向/后向填充，如何实现？

#### 【问题二】 下面的代码实现了什么功能？请仿照设计一个它的groupby版本。

In [351]:
s = pd.Series ([0, 1, 1, 0, 1, 1, 1, 0])
s1 = s.cumsum()
result = s.mul(s1).diff().where(lambda x: x < 0).ffill().add(s1,fill_value =0)
result

0    0.0
1    1.0
2    2.0
3    0.0
4    1.0
5    2.0
6    3.0
7    0.0
dtype: float64

In [356]:
s.mul(s1)

0    0
1    1
2    2
3    0
4    3
5    4
6    5
7    0
dtype: int64

In [359]:
s.mul(s1).diff()

0    NaN
1    1.0
2    1.0
3   -2.0
4    3.0
5    1.0
6    1.0
7   -5.0
dtype: float64

In [363]:
s.mul(s1).diff().where(lambda x: x < 0).ffill()

0    NaN
1    NaN
2    NaN
3   -2.0
4   -2.0
5   -2.0
6   -2.0
7   -5.0
dtype: float64

#### 【问题三】 如何计算组内0.25分位数与0.75分位数？要求显示在同一张表上。

#### 【问题四】 既然索引已经能够选出某些符合条件的子集，那么filter函数的设计有什么意义？

#### 【问题五】 整合、变换、过滤三者在输入输出和功能上有何异同？

#### 【问题六】 在带参数的多函数聚合时，有办法能够绕过wrap技巧实现同样功能吗？

### 2. 练习
#### 【练习一】： 现有一份关于diamonds的数据集，列分别记录了克拉数、颜色、开采深度、价格，请解决下列问题：

In [434]:
#pd.read_csv('data/Diamonds.csv').head()
d=pd.read_csv('data/Diamonds.csv')
d.head()

,carat,color,depth,price
0,0.23,E,61.5,326
1,0.21,E,59.8,326
2,0.23,E,56.9,327
3,0.29,I,62.4,334
4,0.31,J,63.3,335


#### (a) 在所有重量超过1克拉的钻石中，价格的极差是多少？   
  * 极差为 17561    

#### (b) 若以开采深度的0.2\0.4\0.6\0.8分位数为分组依据，每一组中钻石颜色最多的是哪一种？该种颜色是组内平均而言单位重量最贵的吗？
 * 观察发现: (42.999, 60.8]组内, 颜色最多的是E和G,均为2259, 均价最高的是颜色为I的; 
              (60.8, 61.6] 组内, 颜色最多的为G,         均价最高的是颜色为F的.
              (61.6, 62.1] 组内, 颜色最多的为G,         均价最高的是颜色为F的.
              (62.1, 62.7] 组内, 颜色最多的为G,         均价最高的也是颜色为G的.
              (62.7, 79.0] 组内, 颜色最多的为G,         均价最高的也是颜色为G的.

#### (c) 以重量分组(0-0.5,0.5-1,1-1.5,1.5-2,2+)，按递增的深度为索引排序，求每组中连续的严格递增价格序列长度的最大值。
    

#### (d) 请按颜色分组，分别计算价格关于克拉数的回归系数。（单变量的简单线性回归，并只使用Pandas和Numpy完成）
    

In [436]:
#(a) 极差为 17561
d.loc[d.carat>1][['price']].apply(lambda x:x.max()-x.min())

price    17561
dtype: int64

In [437]:
#(b)
bins=[0,0.2,0.4,0.6,0.8,1]
pd.qcut(d['depth'],bins)

0          (60.8, 61.6]
1        (42.999, 60.8]
2        (42.999, 60.8]
3          (62.1, 62.7]
4          (62.7, 79.0]
              ...      
53935    (42.999, 60.8]
53936      (62.7, 79.0]
53937      (62.7, 79.0]
53938      (60.8, 61.6]
53939      (62.1, 62.7]
Name: depth, Length: 53940, dtype: category
Categories (5, interval[float64]): [(42.999, 60.8] < (60.8, 61.6] < (61.6, 62.1] < (62.1, 62.7] < (62.7, 79.0]]

In [438]:
pd.qcut(d['depth'],bins).value_counts()

(60.8, 61.6]      11831
(42.999, 60.8]    11296
(61.6, 62.1]      10403
(62.7, 79.0]      10273
(62.1, 62.7]      10137
Name: depth, dtype: int64

In [432]:
d.groupby(pd.qcut(d['depth'],bins))['color'].describe()#.agg({'price':['count','mean'],'carat':'mean'})#.sort_values(by='price')
# 这样统计出的top只有一个--但是往下看

,count,unique,top,freq
depth,,,,
"(42.999, 60.8]",11296,7,G,2259
"(60.8, 61.6]",11831,7,G,2593
"(61.6, 62.1]",10403,7,G,2247
"(62.1, 62.7]",10137,7,G,2193
"(62.7, 79.0]",10273,7,G,2000


In [454]:
d['avg_price']=d.price/d.carat

In [455]:
d_groupbyqcut=d.groupby([pd.qcut(d['depth'],bins),d.color]).agg({'price':'count','avg_price':'mean'}) # 对单位质量的均价取均值--另一种更合理的选择是, 对组内的价格和质量分别求和再算单价
d_groupbyqcut
# (b) 观察发现: (42.999, 60.8]组内, 颜色最多的是E和G,均为2259, 均价最高的是颜色为I的; 
#              (60.8, 61.6]  组内, 颜色最多的为G,             均价最高的是颜色为F的.
#              (61.6, 62.1]  组内, 颜色最多的为G,             均价最高的是颜色为F的.
#              (62.1, 62.7]  组内, 颜色最多的为G,             均价最高的也是颜色为G的.
#              (62.7, 79.0]  组内, 颜色最多的为G,             均价最高的也是颜色为G的.

price    avg_price
depth          color                    
(42.999, 60.8] D       1519  4096.138305
               E       2259  3929.625897
               F       2060  4136.841550
               G       2259  4294.991176
               H       1597  4275.933161
               I       1063  4452.613168
               J        539  3985.008733
(60.8, 61.6]   D       1526  3920.028256
               E       2175  3695.210838
               F       2184  4125.295966
               G       2593  4119.503438
               H       1783  3882.091164
               I       1052  3808.024196
               J        518  3911.070010
(61.6, 62.1]   D       1233  3955.128801
               E       1895  3661.572571
               F       1876  4152.283895
               G       2247  4040.155790
               H       1559  3887.556283
               I       1068  3840.712944
               J        525  3787.725141
(62.1, 62.7]   D       1202  3928.165051
               E       1641  3929.214233
               F       1763  4204.842133
               G       2193  4258.155489
               H       1652  4058.491162
               I       1095  4020.271378
               J        591  3912.595178
(62.7, 79.0]   D       1295  3842.700843
               E       1827  3816.721165
               F       1659  4050.174162
               G       2000  4106.310203
               H       1713  3950.317729
               I       1144  3868.220357
               J        635  3571.133086

In [456]:
# (c)
d=pd.read_csv('data/Diamonds.csv')
d.head()

,carat,color,depth,price
0,0.23,E,61.5,326
1,0.21,E,59.8,326
2,0.23,E,56.9,327
3,0.29,I,62.4,334
4,0.31,J,63.3,335


In [458]:
bins=[0,0.5,1,1.5,2,np.inf]
pd.cut(d.carat,bins)

0        (0.0, 0.5]
1        (0.0, 0.5]
2        (0.0, 0.5]
3        (0.0, 0.5]
4        (0.0, 0.5]
            ...    
53935    (0.5, 1.0]
53936    (0.5, 1.0]
53937    (0.5, 1.0]
53938    (0.5, 1.0]
53939    (0.5, 1.0]
Name: carat, Length: 53940, dtype: category
Categories (5, interval[float64]): [(0.0, 0.5] < (0.5, 1.0] < (1.0, 1.5] < (1.5, 2.0] < (2.0, inf]]

In [459]:
d.groupby(pd.cut(d.carat,bins)).apply(lambda x:x.sort_values(by='depth'))

carat color  depth  price
carat                                      
(0.0, 0.5] 36503   0.30     E   51.0    945
           40766   0.37     F   52.3   1166
           38041   0.35     D   53.2   1011
           38053   0.34     E   54.0   1012
           37745   0.31     D   54.2    997
...                 ...   ...    ...    ...
(2.0, inf] 13002   2.14     J   69.4   5405
           16439   2.22     H   70.1   6564
           15781   2.01     G   70.2   6315
           17181   2.32     H   70.5   6860
           27646   2.01     G   70.6  18574

[53940 rows x 4 columns]

In [515]:
# (d) 请按颜色分组，分别计算价格关于克拉数的回归系数。（单变量的简单线性回归，并只使用Pandas和Numpy完成）
d=pd.read_csv('data/Diamonds.csv')
d.head()

,carat,color,depth,price
0,0.23,E,61.5,326
1,0.21,E,59.8,326
2,0.23,E,56.9,327
3,0.29,I,62.4,334
4,0.31,J,63.3,335


In [549]:
for name,group in d.groupby('color'):
    mean_price=group['price'].mean()
    #print(mean_price)
    mean_carat=group['carat'].mean()
    #print(mean_carat)
    price=(group['price'].values)
    #print(type(price),len(price))
    carat=(group['carat'].values)
    #print(type(carat),len(carat))
    std_price=np.sqrt((price-mean_price).dot((price-mean_price))/len(group))
    #print(std_price)
    std_carat=np.sqrt((carat-mean_carat).dot((carat-mean_carat))/len(group))
    #print(std_carat)
    #corr_price_carat=
    M=np.vstack((np.ones_like(group.carat.values),group.carat.values)).T
    y=group.price
    p=np.linalg.lstsq(M,y)
    #print(type(p),len(p))
    #print(p)
    print('颜色为{}的分组的截距为{},回归系数为{}'.format(name,p[0][0],p[0][1]))
    

颜色为D的分组的截距为-2361.01715227847,回归系数为8408.353125883328
颜色为E的分组的截距为-2381.0496003790204,回归系数为8296.212783456831
颜色为F的分组的截距为-2665.806190850142,回归系数为8676.658343792675
颜色为G的分组的截距为-2575.5276428585357,回归系数为8525.345779322512
颜色为H的分组的截距为-2460.418046357933,回归系数为7619.09831989575
颜色为I的分组的截距为-2878.1503555754953,回归系数为7761.041168809327
颜色为J的分组的截距为-2920.6033371938993,回归系数为7094.1920922631425


d:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


In [543]:
np.vstack((np.ones_like(d.price.values),d.price.values)).T

array([[   1,  326],
       [   1,  326],
       [   1,  327],
       ...,
       [   1, 2757],
       [   1, 2757],
       [   1, 2757]], dtype=int64)

In [540]:
np.ones_like(d.price.values)

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [536]:
d.price.values

array([ 326,  326,  327, ..., 2757, 2757, 2757], dtype=int64)

In [537]:
np.arange(100)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

#### 【练习二】：有一份关于美国10年至17年的非法药物数据集，列分别记录了年份、州（5个）、县、药物类型、报告数量，请解决下列问题：

In [43]:
pd.read_csv('data/Drugs.csv').head()

,YYYY,State,COUNTY,SubstanceName,DrugReports
0,2010,VA,ACCOMACK,Propoxyphene,1
1,2010,OH,ADAMS,Morphine,9
2,2010,PA,ADAMS,Methadone,2
3,2010,VA,ALEXANDRIA CITY,Heroin,5
4,2010,PA,ALLEGHENY,Hydromorphone,5


#### (a) 按照年份统计，哪个县的报告数量最多？这个县所属的州在当年也是报告数最多的吗？

2010报告数量最多的COUNTY是'PA'state的'PHILADELPHIA',2010报告数量最多的State是'PA'.  
2011报告数量最多的COUNTY是'PA'state的'PHILADELPHIA',2011报告数量最多的State是'OH'.  
[(2012, 'PA', 'PHILADELPHIA')] [(2012, 'OH')]    
[(2013, 'PA', 'PHILADELPHIA')] [(2013, 'OH')]       
[(2014, 'PA', 'PHILADELPHIA')] [(2014, 'OH')]      
[(2015, 'PA', 'PHILADELPHIA')] [(2015, 'OH')]        
[(2016, 'OH', 'HAMILTON')] [(2016, 'OH')]      
[(2017, 'OH', 'HAMILTON')] [(2017, 'OH')]       

#### (b) 从14年到15年，Heroin的数量增加最多的是哪一个州？它在这个州是所有药物中增幅最大的吗？若不是，请找出符合该条件的药物。
* 从2014到2015,Heroin增长最多的State 是 OH. 从2014到2015, OH增幅最大的是 Acetyl fentanyl.

In [461]:
d=pd.read_csv('data/Drugs.csv')
print(d.shape)
d.head()

(24062, 5)


,YYYY,State,COUNTY,SubstanceName,DrugReports
0,2010,VA,ACCOMACK,Propoxyphene,1
1,2010,OH,ADAMS,Morphine,9
2,2010,PA,ADAMS,Methadone,2
3,2010,VA,ALEXANDRIA CITY,Heroin,5
4,2010,PA,ALLEGHENY,Hydromorphone,5


In [486]:
d.groupby([d.YYYY,d.State,d.COUNTY]).sum()#.sort_values(by='DrugReports')

DrugReports
YYYY State COUNTY               
2010 KY    ADAIR               7
           ALLEN              30
           ANDERSON           34
           BALLARD             1
           BARREN             62
...                          ...
2017 WV    WAYNE              26
           WEBSTER            12
           WETZEL              3
           WOOD              242
           WYOMING             1

[3480 rows x 1 columns]

In [495]:
# 2010年报告数最多的COUNTY为PHILADELPHIA, 其所在State为PA
d.groupby([d.YYYY,d.State,d.COUNTY]).sum().loc[pd.IndexSlice[2010,:,:],:].idxmax().values

array([(2010, 'PA', 'PHILADELPHIA')], dtype=object)

In [496]:
# 2010年报告数最多的State为PA
d.groupby([d.YYYY,d.State]).sum().loc[pd.IndexSlice[2010,:],:].idxmax().values

array([(2010, 'PA')], dtype=object)

In [498]:
for i in range(d.YYYY.min(),d.YYYY.max()+1):
    #print(i)
    print(d.groupby([d.YYYY,d.State,d.COUNTY]).sum().loc[pd.IndexSlice[i,:,:],:].idxmax().values)
    print(d.groupby([d.YYYY,d.State]).sum().loc[pd.IndexSlice[i,:],:].idxmax().values)
# (a)

[(2010, 'PA', 'PHILADELPHIA')]
[(2010, 'PA')]
[(2011, 'PA', 'PHILADELPHIA')]
[(2011, 'OH')]
[(2012, 'PA', 'PHILADELPHIA')]
[(2012, 'OH')]
[(2013, 'PA', 'PHILADELPHIA')]
[(2013, 'OH')]
[(2014, 'PA', 'PHILADELPHIA')]
[(2014, 'OH')]
[(2015, 'PA', 'PHILADELPHIA')]
[(2015, 'OH')]
[(2016, 'OH', 'HAMILTON')]
[(2016, 'OH')]
[(2017, 'OH', 'HAMILTON')]
[(2017, 'OH')]


In [509]:
d.loc[d.SubstanceName=='Heroin'][d.YYYY==2014].groupby('State')['DrugReports'].sum()

d:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


State
KY     4362
OH    20590
PA    18454
VA     3132
WV     1516
Name: DrugReports, dtype: int64

In [510]:
d.loc[d.SubstanceName=='Heroin'][d.YYYY==2015].groupby('State')['DrugReports'].sum()

d:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


State
KY     4045
OH    23347
PA    18964
VA     3583
WV     1135
Name: DrugReports, dtype: int64

In [511]:
d.loc[d.SubstanceName=='Heroin'][d.YYYY==2015].groupby('State')['DrugReports'].sum()-d.loc[d.SubstanceName=='Heroin'][d.YYYY==2014].groupby('State')['DrugReports'].sum()
# (b) Heroin增长最多的State 是 OH

d:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


State
KY    -317
OH    2757
PA     510
VA     451
WV    -381
Name: DrugReports, dtype: int64

In [514]:
(d.loc[d.State=='OH'][d.YYYY==2015].groupby('SubstanceName')['DrugReports'].sum()-d.loc[d.State=='OH'][d.YYYY==2014].groupby('SubstanceName')['DrugReports'].sum())\
/d.loc[d.State=='OH'][d.YYYY==2014].groupby('SubstanceName')['DrugReports'].sum()
# (b) 从2014到2015,OH增幅最大的是 Acetyl fentanyl.

d:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
d:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


SubstanceName
ANPP                              NaN
Acetyl fentanyl             14.500000
Buprenorphine                0.117864
Butorphanol                       NaN
Butyryl fentanyl                  NaN
Codeine                      0.005376
Dextropropoxyphene                NaN
Dihydrocodeine                    NaN
Fentanyl                     1.950480
Furanyl fentanyl                  NaN
Heroin                       0.133900
Hydrocodone                 -0.111504
Hydromorphone               -0.086420
Meperidine                  -0.333333
Methadone                   -0.126316
Mitragynine                 -0.200000
Morphine                     0.149457
Oxycodone                    0.027817
Oxymorphone                  0.246032
Pentazocine                 -0.800000
Propoxyphene                 0.636364
Thebaine                          NaN
Tramadol                     1.451362
p-Fluorobutyryl fentanyl          NaN
Name: DrugReports, dtype: float64